# Assignment 1 - Part A and B:
In Part A you will learn how to code Belief Propogation. For Part B you have to compare different landcover classification approaches. How to load the data is discussed in this notebook.

**PLEASE USE `scikit-learn` FOR ALL MACHINE LEARNING RELATED TASKS**

**YOU ARE ALWAYS REQUIRED TO SUBMIT THE CODE THAT YOU DEVELOPED ALONGSIDE THE COMPLETED ANSWER NOTEBOOK**


**You will also be required to report on all the contents of this notebook once completed, the results sould be in your report and well documented**

### Group Name:
### Student Number(s):

This notebook is divided into two parts. You will first be required to implement belief propogation.
You will then be expected to build a sequential classifier (and other classification models) which can discern between vegatation and settlement MODIS pixels. 

# Part A: Belief Propogation

Implement a BPSK (Binary Phase Shift Keying) modulation scheme on an AWGN (Additive White Gaussian Noise) communication channel.

Use this implementation to plot the simulated BER curve associated with an uncoded BPSK AWGN channel. 

Also plot the theoretical BER curve for a uncoded BPSK channel. 

Now add a Hamming (7,4,3) code to the channel and decode it using belief propogation. 

Also plot the BER curve of the coded BPSK AWGN channel.


_Make sure that the resulting curve is very smooth, otherwise marks will be deducted_

In [ ]:
from IPython.display import Image
Image("./uncoded_BPSK.jpg")

Your plot should look similar to:

In [ ]:
from IPython.display import Image
Image("./ber.jpg")

In [ ]:
"""Code Here"""





# Part B Remote Sensing - MODIS 

How to load the dataset you will be using for part B of the assignment is discussed below.

In [ ]:
import scipy.io
import pylab as plt
import numpy as np

## Plotting a random vegetation and settlement MODIS pixel

In [ ]:
def plot_mat(name=""):
    mat = scipy.io.loadmat(name)
    print(mat.keys())
    veg_gauteng = mat['veg_gauteng']
          
    #(time,pixels,band)
    #(time,pixels,7) - NDVI
    print(veg_gauteng.shape)

    #(time,pixels,band)
    #(time,pixels,7) - NDVI
    bwt_gauteng = mat['bwt_gauteng']
    #bldg_gauteng = mat['bldg_gauteng']
    print(bwt_gauteng.shape)
                   
    #PLOT A SINGLE VEGETATION MODIS PIXEL (over all time and bands - except NDVI) 
    for k in range(7):
          plt.plot(veg_gauteng[:,0,k])
    plt.title("VEGETATION")
    plt.show()
          
    #PLOT A SINGLE SETTLEMENT MODIS PIXEL (over all time and bands - except NDVI) 
    for k in range(7):
        plt.plot(bwt_gauteng[:,0,k])

    plt.title("SETTLEMENT") 
    plt.show()      

## Load the data set

In this part of the assignment you will be required to train a time-varying MODIS classification model. You will then use the SPRT algorithm and the trained time-varying model to perform sequential classification of MODIS time-series. You will also implement a Naive Bayes classifier and a Logistic Regression classifier that takes as input the MODIS data after having performed dimensionality reduction on it. In the end you will compare the efficacy of the different classification algorithms with one another. Your main task is to discern between vegetation and settlement pixels. Below we plot a random MODIS vegetation and settlement pixel.

In [ ]:
plot_mat(name="Gauteng_nochange.mat")

We now load the data into a multi-dimensional numpy array.

In [ ]:
mat = scipy.io.loadmat("Gauteng_nochange.mat")
veg_gauteng = mat['veg_gauteng']
bwt_gauteng = mat['bwt_gauteng']

**MISSING ANSWER**

Please plot the data recorded at time-step 7 of 45 in the first year. Your result should look similar to the graph depicted below.

In [ ]:
Image("./sample.png")

**`MISSING CODE`**

Let's create our observation matrix $\mathbf{X}$ -- [samples, features] and our class labels $\mathbf{y}$.

In [ ]:
X = np.concatenate((veg_gauteng,bwt_gauteng),axis=1)
print(X.shape)
yv = np.ones((592,1),dtype=int)
yb = np.zeros((333,1),dtype=int)
y = np.vstack((yv,yb))[:,0]

print(X.shape)
print(y.shape)

Now that you have your data and you understand it it is time to complete Part B of the assignment. Please refer to the main assignment document for the details.

In [ ]:
"""Code Here if you choose to answer Part B in the notebook"""